In [1]:
!nvidia-smi

Wed Sep  6 14:16:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   35C    P0    58W / 300W |  31956MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6'

In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
import pandas as pd
import numpy as np
import requests, string, re, collections, math
from tqdm import tqdm
from transformers import pipeline

# Download output of baseline

In [5]:
def convert_question_and_answer_to_hypothesis(data):
    for i in tqdm(range(len(data))):
        data['Prediction Hypothesis'] = data['Question'] + ' ' + data['Prediction Answer']
        data['Gold Hypothesis'] = data['Question'] + ' ' + data['Gold Answer']
    return data

In [6]:
def normalize_text(s):
    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)
    def white_space_fix(text):
        return " ".join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def return_acc_and_f1(data, message):
    em = 0
    f1_arr = []
    
    for i in tqdm(range(len(data))):
        
        pred = str(data['Prediction Answer'][i])
        gold = str(data['Gold Answer'][i])
        
        # for exact match
        if pred == gold:
            em += 1
        
        # for f1
        pred_tokens = normalize_text(pred).split()
        gold_tokens = normalize_text(gold).split()
        common = collections.Counter(pred_tokens) & collections.Counter(gold_tokens)
        num_same = sum(common.values()) # True positive

        if len(gold_tokens) == 0 or len(pred_tokens) == 0: 
            f1 = int(gold_tokens == pred_tokens)
            f1_arr.append(f1)
            continue

        if num_same == 0:
            f1 = 0
            f1_arr.append(f1)
            continue

        precision = 1.0 * num_same / len(pred_tokens)
        recall = 1.0 * num_same / len(gold_tokens)
        f1 = (2.0 * precision * recall) / (precision + recall)
        f1_arr.append(f1)
        
    em_final = em / len(data)
    f1_final = np.mean(f1_arr)
    
    print(message.upper())
    print("Final exact match:", round(em_final, 3))
    print("Average F1 score:", round(f1_final, 3))

In [7]:
df_squadid = pd.read_csv("output_squadid_df.csv")
df_squadid = df_squadid[["Context", "Question", "Prediction Answer", "Gold Answer"]]
df_squadid = convert_question_and_answer_to_hypothesis(df_squadid)
df_squadid = df_squadid.dropna(subset=['Prediction Answer', 'Gold Answer']).reset_index(drop=True)
print(df_squadid['Prediction Answer'].isna().sum())
df_squadid

100%|████████████████████████████████████████████████████████████████████████████| 11962/11962 [01:18<00:00, 152.65it/s]

0


,Context,Question,Prediction Answer,Gold Answer,Prediction Hypothesis,Gold Hypothesis
0,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Di negara apa Normandia berada?,Perancis,Perancis,Di negara apa Normandia berada? Perancis,Di negara apa Normandia berada? Perancis
1,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Kapan Normandia di Normandia?,abad ke-10 dan ke-11,-10 dan ke,Kapan Normandia di Normandia? abad ke-10 dan k...,Kapan Normandia di Normandia? -10 dan ke
2,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Dari negara mana asal Norse?,"Denmark, Islandia dan Norwegia","Denmark, Islandia dan Norwegia","Dari negara mana asal Norse? Denmark, Islandia...","Dari negara mana asal Norse? Denmark, Islandia..."
3,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Siapa pemimpin Norse?,Rollo,Rollo,Siapa pemimpin Norse? Rollo,Siapa pemimpin Norse? Rollo
4,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Abad berapa pertama kali Normandia mendapatkan...,abad ke-10,abad ke-10,Abad berapa pertama kali Normandia mendapatkan...,Abad berapa pertama kali Normandia mendapatkan...
...,...,...,...,...,...,...
10499,Hubungan antara kekuatan nonkonservatif makros...,Kekuatan konservatif sering dikaitkan dengan p...,panas,panas,Kekuatan konservatif sering dikaitkan dengan p...,Kekuatan konservatif sering dikaitkan dengan p...
10500,"Gaya pon memiliki padanan metrik, yang lebih j...",Seperti apa kekuatan kilogram yang kadang-kada...,kilopond,kilopond,Seperti apa kekuatan kilogram yang kadang-kada...,Seperti apa kekuatan kilogram yang kadang-kada...
10501,"Gaya pon memiliki padanan metrik, yang lebih j...",Apakah satuan massa yang sangat jarang digunak...,newton,siput,Apakah satuan massa yang sangat jarang digunak...,Apakah satuan massa yang sangat jarang digunak...
10502,"Gaya pon memiliki padanan metrik, yang lebih j...",Apa yang jarang menggunakan istilah satuan kek...,sthène,kip,Apa yang jarang menggunakan istilah satuan kek...,Apa yang jarang menggunakan istilah satuan kek...


In [8]:
return_acc_and_f1(df_squadid, "squad-id")

100%|██████████████████████████████████████████████████████████████████████████| 10504/10504 [00:00<00:00, 27310.38it/s]

SQUAD-ID
Final exact match: 0.558
Average F1 score: 0.753


In [9]:
url = "https://huggingface.co/muhammadravi251001/fine-tuned-DatasetQAS-IDK-MRC-with-xlm-roberta-large-without-ITTL-without-freeze-LR-1e-05/raw/main/results/output/output_df.csv"
response = requests.get(url)

if response.status_code == 200:
    with open("data.csv", "wb") as file:
        file.write(response.content)
    df_idkmrc = pd.read_csv("data.csv")
else: print("Download failed!")
    
df_idkmrc = df_idkmrc[["Context", "Question", "Prediction Answer", "Gold Answer"]]
df_idkmrc = convert_question_and_answer_to_hypothesis(df_idkmrc)
df_idkmrc = df_idkmrc.dropna(subset=['Prediction Answer', 'Gold Answer']).reset_index(drop=True)
print(df_idkmrc['Prediction Answer'].isna().sum())
df_idkmrc

100%|████████████████████████████████████████████████████████████████████████████████| 848/848 [00:01<00:00, 696.99it/s]

0


,Context,Question,Prediction Answer,Gold Answer,Prediction Hypothesis,Gold Hypothesis
0,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan Komputer mikro mulai dikembangkan?,1959,1959,Kapan Komputer mikro mulai dikembangkan? 1959,Kapan Komputer mikro mulai dikembangkan? 1959
1,"Patronim, atau patronimik, adalah sebuah kompo...",Apakah pengertian matronimik?,komponen dari sebuah nama pribadi yang berdasa...,Komomene sebuah nama yang berdasarkan pada nam...,Apakah pengertian matronimik? komponen dari se...,Apakah pengertian matronimik? Komomene sebuah ...
2,"Ir. Basuki Tjahaja Purnama, M.M. (EYD: Basuki ...",Siapakah Basuki Tjahaja Purnama?,Gubernur DKI Jakarta,Gubernur DKI Jakarta yang menjabat sejak 19 No...,Siapakah Basuki Tjahaja Purnama? Gubernur DKI ...,Siapakah Basuki Tjahaja Purnama? Gubernur DKI ...
3,"Ir. Basuki Tjahaja Purnama, M.M. (EYD: Basuki ...",Siapakah Gubernur DKI Jakarta yang menjabat se...,Ir. Basuki Tjahaja Purnama,Basuki Tjahaja Purnama,Siapakah Gubernur DKI Jakarta yang menjabat se...,Siapakah Gubernur DKI Jakarta yang menjabat se...
4,Setelah kepala Sebastianus dan Jovinus tiba di...,Kapan Raja Ataulf menikah?,Januari 414,Januari 414,Kapan Raja Ataulf menikah? Januari 414,Kapan Raja Ataulf menikah? Januari 414
...,...,...,...,...,...,...
374,Studi hubungan internasional sebagai teori sud...,Kapan Teori hubungan internasional diciptakan?,1939,1939,Kapan Teori hubungan internasional diciptakan?...,Kapan Teori hubungan internasional diciptakan?...
375,"Dalam melaksanakan kegiatan belajar-mengajar, ...",Berapa luas SMK Negeri 1 Cikampek?,28997m2,29095m2.,Berapa luas SMK Negeri 1 Cikampek? 28997m2,Berapa luas SMK Negeri 1 Cikampek? 29095m2.
376,Ikan pari manta (Manta birostris) adalah salah...,Berapakah berat Ikan pari manta yag terbesar?,3 ton,3 ton,Berapakah berat Ikan pari manta yag terbesar? ...,Berapakah berat Ikan pari manta yag terbesar? ...
377,Kota ini terletak di sebuah lembah sungai yang...,"Menurut Biro Sensus Amerika Serikat, berapa lu...",131.3 mil persegi,"Menurut Biro Sensus Amerika Serikat, kota ini ...","Menurut Biro Sensus Amerika Serikat, berapa lu...","Menurut Biro Sensus Amerika Serikat, berapa lu..."


In [10]:
return_acc_and_f1(df_idkmrc, "idk-mrc")

100%|██████████████████████████████████████████████████████████████████████████████| 379/379 [00:00<00:00, 21144.75it/s]

IDK-MRC
Final exact match: 0.704
Average F1 score: 0.854


In [11]:
url = "https://huggingface.co/muhammadravi251001/fine-tuned-DatasetQAS-TYDI-QA-ID-with-xlm-roberta-large-without-ITTL-without-freeze-LR-1e-05/raw/main/results/output/output_df.csv"
response = requests.get(url)

if response.status_code == 200:
    with open("data.csv", "wb") as file:
        file.write(response.content)
    df_tydiqaid = pd.read_csv("data.csv")
else: print("Download failed!")
    
df_tydiqaid = df_tydiqaid[["Context", "Question", "Prediction Answer", "Gold Answer"]]
df_tydiqaid = convert_question_and_answer_to_hypothesis(df_tydiqaid)
df_tydiqaid = df_tydiqaid.dropna(subset=['Prediction Answer', 'Gold Answer']).reset_index(drop=True)
print(df_tydiqaid['Prediction Answer'].isna().sum())
df_tydiqaid

100%|████████████████████████████████████████████████████████████████████████████████| 857/857 [00:01<00:00, 645.26it/s]

0


,Context,Question,Prediction Answer,Gold Answer,Prediction Hypothesis,Gold Hypothesis
0,Ernest Douwes Dekker wafat dini hari tanggal 2...,dimanakah Dr. Ernest François Eugène Douwes De...,"TMP Cikutra, Bandung",28 Agustus 1950,dimanakah Dr. Ernest François Eugène Douwes De...,dimanakah Dr. Ernest François Eugène Douwes De...
1,"Pada tanggal 18 Februari 2008, desain Yoo Kerl...",Siapa arsitek Balai Kota Seoul?,Yoo Kerl,Yoo Kerl,Siapa arsitek Balai Kota Seoul? Yoo Kerl,Siapa arsitek Balai Kota Seoul? Yoo Kerl
2,Sebagai tindak lanjut Atlantic Charter tersebu...,Kapan PBB mulai terbentuk?,24 Oktober 1945,24 Oktober 1945,Kapan PBB mulai terbentuk? 24 Oktober 1945,Kapan PBB mulai terbentuk? 24 Oktober 1945
3,"Dia dipenjarakan di Puri Dragsholm, 75 kilomet...",Dimana James Hepburn meninggal?,"Puri Dragsholm, 75 kilometer Kopenhagen",Puri Dragsholm,Dimana James Hepburn meninggal? Puri Dragsholm...,Dimana James Hepburn meninggal? Puri Dragsholm
4,"Lahir di Sheffield, South Yorkshire, Vardy mem...",Dimana Jamie Richard Vardy lahir?,"Sheffield, South Yorkshire","Sheffield, South Yorkshire","Dimana Jamie Richard Vardy lahir? Sheffield, S...","Dimana Jamie Richard Vardy lahir? Sheffield, S..."
...,...,...,...,...,...,...
830,Raden Patah (Jawa: code: jav promoted to code:...,Siapa raja Demak pertama?,Raden Patah,Raden Patah,Siapa raja Demak pertama? Raden Patah,Siapa raja Demak pertama? Raden Patah
831,Laut dalam adalah lapisan terbawah dari lautan...,Berapakah kedalaman laut yang disebut dengan l...,1828 m,1828 m,Berapakah kedalaman laut yang disebut dengan l...,Berapakah kedalaman laut yang disebut dengan l...
832,"Justus Heurnius (lahir di Utrecht, Belanda, 15...",Kapan Justus Heurnius lahir?,1587,1587,Kapan Justus Heurnius lahir? 1587,Kapan Justus Heurnius lahir? 1587
833,Frekuensi suara atau frekuensi audio yaitu get...,Apakah yang dimaksud dengan frekuensi audio?,getaran frekuensi yang terdengar oleh manusia ...,getaran frekuensi yang terdengar oleh manusia ...,Apakah yang dimaksud dengan frekuensi audio? g...,Apakah yang dimaksud dengan frekuensi audio? g...


In [12]:
return_acc_and_f1(df_tydiqaid, "tydi-qa-id")

100%|██████████████████████████████████████████████████████████████████████████████| 835/835 [00:00<00:00, 20053.62it/s]

TYDI-QA-ID
Final exact match: 0.683
Average F1 score: 0.803


# Test the hypothesis above with newest NLI model

In [13]:
tokenizer_kwargs = {'truncation': True, 'max_length': 512}
model_nli_name = "muhammadravi251001/fine-tuned-NLI-idk-mrc-nli-keep-with-xlm-roberta-large"
nli_model = pipeline("text-classification", model=model_nli_name, tokenizer=model_nli_name, **tokenizer_kwargs)

In [14]:
nli_model({'text': "Bambang Pamungkas seorang pemain bola asal Jakarta", 
           'text_pair': "Bambang Pamungkas berasal dari Jakarta"})

{'label': 'entailment', 'score': 0.9963834285736084}

In [15]:
nli_model({'text': "Bambang Pamungkas seorang pemain bola asal Jakarta", 
           'text_pair': "Bambang Pamungkas bukan seorang pemain bola"})

{'label': 'entailment', 'score': 0.9935256242752075}

In [16]:
nli_model({'text': "Bambang Pamungkas seorang pemain bola asal Jakarta", 
           'text_pair': "Bambang Pamungkas berasal dari Bandung"})

{'label': 'entailment', 'score': 0.9755388498306274}

In [17]:
def add_label(data, message, nli_model=nli_model):
    
    data["Label from Prediction Answer"] = str()
    data["Label from Gold Answer"] = str()
    
    labels_pred_answer = []
    labels_gold_answer = []
    
    for i in tqdm(range(len(data))):
        
        premise = data['Context'][i]
        pred_hypo = data['Prediction Hypothesis'][i]
        gold_hypo = data['Gold Hypothesis'][i]
        
        label_pred_answer = nli_model({'text': premise, 'text_pair': pred_hypo})['label']
        labels_pred_answer.append(label_pred_answer)
        
        label_gold_answer = nli_model({'text': premise, 'text_pair': gold_hypo})['label']
        labels_gold_answer.append(label_gold_answer)
    
    data["Label from Prediction Answer"] = labels_pred_answer
    data["Label from Gold Answer"] = labels_gold_answer
    
    total_entailment_gold = data['Label from Gold Answer'].str.count('entailment').sum()
    total_neutral_gold = data['Label from Gold Answer'].str.count('neutral').sum()
    total_contradiction_gold = data['Label from Gold Answer'].str.count('contradiction').sum()
    
    total_entailment_pred = data['Label from Prediction Answer'].str.count('entailment').sum()
    total_neutral_pred = data['Label from Prediction Answer'].str.count('neutral').sum()
    total_contradiction_pred = data['Label from Prediction Answer'].str.count('contradiction').sum()
    
    total_label_gold = len(data[data['Label from Gold Answer'] != "NULL"])
    total_label_pred = len(data[data['Label from Prediction Answer'] != "NULL"])
    
    print(f"PREDICTION {message.upper()}")
    print(f"Total entailment: {total_entailment_pred} ({round(total_entailment_pred/total_label_pred, 2) * 100} %)")
    print(f"Total neutral: {total_neutral_pred} ({round(total_neutral_pred/total_label_pred, 2) * 100} %)")
    print(f"Total contradiction: {total_contradiction_pred} ({round(total_contradiction_pred/total_label_pred, 2) * 100} %)")
    print()
    print(f"GOLD {message.upper()}")
    print(f"Total entailment: {total_entailment_gold} ({round(total_entailment_gold/total_label_gold, 2) * 100} %)")
    print(f"Total neutral: {total_neutral_gold} ({round(total_neutral_gold/total_label_gold, 2) * 100} %)")
    print(f"Total contradiction: {total_contradiction_gold} ({round(total_contradiction_gold/total_label_gold, 2) * 100} %)")
    print()
    print(f"Total data: {len(data)}")
    
    return data

In [18]:
print(df_idkmrc['Context'][16])
print(df_idkmrc['Prediction Hypothesis'][16])
print()
print(df_idkmrc['Context'][17])
print(df_idkmrc['Prediction Hypothesis'][17])
print()
print(df_idkmrc['Context'][18])
print(df_idkmrc['Prediction Hypothesis'][18])

Bidara diperkirakan memiliki asal usul dari Asia Tengah, dan menyebar alami di wilayah yang luas mulai dari Aljazair, Tunisia, Libia, Mesir, Uganda dan Kenya di Afrika; Afganistan, Pakistan, India utara, Nepal, Bangladesh, Cina selatan, Vietnam, Thailand, Semenanjung Malaya, Indonesia, hingga Australia. Kini bidara telah ditanam di banyak negara di Afrika, dan juga di Madagaskar.[4] Namun yang mengembangkannya secara komersial hanyalah India, Cina, dan sedikit di Thailand[2].
dari manakah asal pohon Bidara? Asia Tengah

Namun berkat bantuan para pengikutnya dan masyarakat Pulau Banda yang bersimpati kepada Wong Agung Wilis, ia akhirnya bisa lolos dari Penjara Rosingain (tempatnya diasingkan) dan lari ke Pulau Seram dan kemudian berlayar ke Bali. Sesampainya di Bali, ia tidak melanjutkan perjuangan. Meskipun keturunan-keturunan dan pengikut-pengikutnya (Seperti Pangeran Jagapati yang terkenal pada Perang Puputan Bayu pada tahun 1773) tetap berjuang melawan VOC walau tidak berhasil. Ia m

In [19]:
df_idkmrc = add_label(df_idkmrc, "idk-mrc")
df_idkmrc

100%|█████████████████████████████████████████████████████████████████████████████████| 379/379 [10:08<00:00,  1.61s/it]

PREDICTION <built-in method upper of str object at 0x7fef43d980f0>
Total entailment: 371 (98.0 %)
Total neutral: 0 (0.0 %)
Total contradiction: 8 (2.0 %)

GOLD <built-in method upper of str object at 0x7fef43d980f0>
Total entailment: 373 (98.0 %)
Total neutral: 0 (0.0 %)
Total contradiction: 6 (2.0 %)

Total data: 379


,Context,Question,Prediction Answer,Gold Answer,Prediction Hypothesis,Gold Hypothesis,Label from Prediction Answer,Label from Gold Answer
0,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan Komputer mikro mulai dikembangkan?,1959,1959,Kapan Komputer mikro mulai dikembangkan? 1959,Kapan Komputer mikro mulai dikembangkan? 1959,entailment,entailment
1,"Patronim, atau patronimik, adalah sebuah kompo...",Apakah pengertian matronimik?,komponen dari sebuah nama pribadi yang berdasa...,Komomene sebuah nama yang berdasarkan pada nam...,Apakah pengertian matronimik? komponen dari se...,Apakah pengertian matronimik? Komomene sebuah ...,entailment,entailment
2,"Ir. Basuki Tjahaja Purnama, M.M. (EYD: Basuki ...",Siapakah Basuki Tjahaja Purnama?,Gubernur DKI Jakarta,Gubernur DKI Jakarta yang menjabat sejak 19 No...,Siapakah Basuki Tjahaja Purnama? Gubernur DKI ...,Siapakah Basuki Tjahaja Purnama? Gubernur DKI ...,entailment,entailment
3,"Ir. Basuki Tjahaja Purnama, M.M. (EYD: Basuki ...",Siapakah Gubernur DKI Jakarta yang menjabat se...,Ir. Basuki Tjahaja Purnama,Basuki Tjahaja Purnama,Siapakah Gubernur DKI Jakarta yang menjabat se...,Siapakah Gubernur DKI Jakarta yang menjabat se...,entailment,entailment
4,Setelah kepala Sebastianus dan Jovinus tiba di...,Kapan Raja Ataulf menikah?,Januari 414,Januari 414,Kapan Raja Ataulf menikah? Januari 414,Kapan Raja Ataulf menikah? Januari 414,entailment,entailment
...,...,...,...,...,...,...,...,...
374,Studi hubungan internasional sebagai teori sud...,Kapan Teori hubungan internasional diciptakan?,1939,1939,Kapan Teori hubungan internasional diciptakan?...,Kapan Teori hubungan internasional diciptakan?...,entailment,entailment
375,"Dalam melaksanakan kegiatan belajar-mengajar, ...",Berapa luas SMK Negeri 1 Cikampek?,28997m2,29095m2.,Berapa luas SMK Negeri 1 Cikampek? 28997m2,Berapa luas SMK Negeri 1 Cikampek? 29095m2.,entailment,entailment
376,Ikan pari manta (Manta birostris) adalah salah...,Berapakah berat Ikan pari manta yag terbesar?,3 ton,3 ton,Berapakah berat Ikan pari manta yag terbesar? ...,Berapakah berat Ikan pari manta yag terbesar? ...,entailment,entailment
377,Kota ini terletak di sebuah lembah sungai yang...,"Menurut Biro Sensus Amerika Serikat, berapa lu...",131.3 mil persegi,"Menurut Biro Sensus Amerika Serikat, kota ini ...","Menurut Biro Sensus Amerika Serikat, berapa lu...","Menurut Biro Sensus Amerika Serikat, berapa lu...",entailment,entailment


In [20]:
df_tydiqaid = add_label(df_tydiqaid, "tydi-qa-id")
df_tydiqaid

100%|█████████████████████████████████████████████████████████████████████████████████| 835/835 [22:07<00:00,  1.59s/it]

PREDICTION <built-in method upper of str object at 0x7fef43d13630>
Total entailment: 828 (99.0 %)
Total neutral: 0 (0.0 %)
Total contradiction: 7 (1.0 %)

GOLD <built-in method upper of str object at 0x7fef43d13630>
Total entailment: 828 (99.0 %)
Total neutral: 0 (0.0 %)
Total contradiction: 7 (1.0 %)

Total data: 835


,Context,Question,Prediction Answer,Gold Answer,Prediction Hypothesis,Gold Hypothesis,Label from Prediction Answer,Label from Gold Answer
0,Ernest Douwes Dekker wafat dini hari tanggal 2...,dimanakah Dr. Ernest François Eugène Douwes De...,"TMP Cikutra, Bandung",28 Agustus 1950,dimanakah Dr. Ernest François Eugène Douwes De...,dimanakah Dr. Ernest François Eugène Douwes De...,entailment,entailment
1,"Pada tanggal 18 Februari 2008, desain Yoo Kerl...",Siapa arsitek Balai Kota Seoul?,Yoo Kerl,Yoo Kerl,Siapa arsitek Balai Kota Seoul? Yoo Kerl,Siapa arsitek Balai Kota Seoul? Yoo Kerl,entailment,entailment
2,Sebagai tindak lanjut Atlantic Charter tersebu...,Kapan PBB mulai terbentuk?,24 Oktober 1945,24 Oktober 1945,Kapan PBB mulai terbentuk? 24 Oktober 1945,Kapan PBB mulai terbentuk? 24 Oktober 1945,entailment,entailment
3,"Dia dipenjarakan di Puri Dragsholm, 75 kilomet...",Dimana James Hepburn meninggal?,"Puri Dragsholm, 75 kilometer Kopenhagen",Puri Dragsholm,Dimana James Hepburn meninggal? Puri Dragsholm...,Dimana James Hepburn meninggal? Puri Dragsholm,entailment,entailment
4,"Lahir di Sheffield, South Yorkshire, Vardy mem...",Dimana Jamie Richard Vardy lahir?,"Sheffield, South Yorkshire","Sheffield, South Yorkshire","Dimana Jamie Richard Vardy lahir? Sheffield, S...","Dimana Jamie Richard Vardy lahir? Sheffield, S...",entailment,entailment
...,...,...,...,...,...,...,...,...
830,Raden Patah (Jawa: code: jav promoted to code:...,Siapa raja Demak pertama?,Raden Patah,Raden Patah,Siapa raja Demak pertama? Raden Patah,Siapa raja Demak pertama? Raden Patah,entailment,entailment
831,Laut dalam adalah lapisan terbawah dari lautan...,Berapakah kedalaman laut yang disebut dengan l...,1828 m,1828 m,Berapakah kedalaman laut yang disebut dengan l...,Berapakah kedalaman laut yang disebut dengan l...,entailment,entailment
832,"Justus Heurnius (lahir di Utrecht, Belanda, 15...",Kapan Justus Heurnius lahir?,1587,1587,Kapan Justus Heurnius lahir? 1587,Kapan Justus Heurnius lahir? 1587,entailment,entailment
833,Frekuensi suara atau frekuensi audio yaitu get...,Apakah yang dimaksud dengan frekuensi audio?,getaran frekuensi yang terdengar oleh manusia ...,getaran frekuensi yang terdengar oleh manusia ...,Apakah yang dimaksud dengan frekuensi audio? g...,Apakah yang dimaksud dengan frekuensi audio? g...,entailment,entailment


In [ ]:
df_squadid = add_label(df_squadid, "squad-id")
df_squadid

  3%|██▋                                                                          | 359/10504 [11:14<7:36:28,  2.70s/it]